In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [2]:
weather_df = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/weather_data/DWD_ICON-EU.csv')
solar_total = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/solar_total_production.csv')

In [3]:
weather_df.sort_values(by='ref_datetime', inplace=True)

In [4]:
weather_df = weather_df.groupby(["valid_datetime","latitude","longitude"]).last().reset_index()

In [5]:
print(weather_df.columns)

Index(['valid_datetime', 'latitude', 'longitude', 'ref_datetime',
       'Temperature', 'WindSpeed', 'WindSpeed:100', 'WindDirection:100',
       'CloudCover', 'RelativeHumidity', 'PressureReducedMSL',
       'SolarDownwardRadiation', 'TotalPrecipitation'],
      dtype='object')


In [6]:
weather_df.reset_index(inplace=True)

In [7]:
def resample_and_interpolate(group):
    return group.reset_index(level=[1, 2]).resample('30T').asfreq().interpolate()

In [8]:
weather_df.valid_datetime = pd.to_datetime(weather_df.valid_datetime)
weather_df = weather_df.set_index(["valid_datetime","latitude","longitude"])
df_resampled = weather_df.groupby(['latitude', 'longitude'], group_keys=False).apply(resample_and_interpolate)

C:\Users\paulh\AppData\Local\Temp\ipykernel_2168\2834871732.py:2: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  return group.reset_index(level=[1, 2]).resample('30T').asfreq().interpolate()
C:\Users\paulh\AppData\Local\Temp\ipykernel_2168\2834871732.py:2: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  return group.reset_index(level=[1, 2]).resample('30T').asfreq().interpolate()
C:\Users\paulh\AppData\Local\Temp\ipykernel_2168\2834871732.py:2: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  return group.reset_index(level=[1, 2]).resample('30T').asfreq().interpolate()
C:\Users\paulh\AppData\Local\Temp\ipykernel_2168\283

In [9]:
df_resampled = df_resampled.reset_index()

In [10]:
solar_total.timestamp_utc = pd.to_datetime(solar_total.timestamp_utc)

In [11]:
df_resampled.drop(columns=['index','ref_datetime'], inplace=True)

In [132]:
# df_resampled = df_resampled.groupby("valid_datetime").mean().reset_index()

In [12]:
df_resampled_merged = pd.merge(df_resampled, solar_total, how='left', left_on='valid_datetime', right_on='timestamp_utc')

In [13]:
df_resampled_merged.columns

Index(['valid_datetime', 'latitude', 'longitude', 'Temperature', 'WindSpeed',
       'WindSpeed:100', 'WindDirection:100', 'CloudCover', 'RelativeHumidity',
       'PressureReducedMSL', 'SolarDownwardRadiation', 'TotalPrecipitation',
       'timestamp_utc', 'generation_mw', 'installed_capacity_mwp',
       'capacity_mwp'],
      dtype='object')

solar

In [14]:
df_resampled_merged_solar = df_resampled_merged.loc[~(df_resampled_merged.latitude == 53.935) & ~(df_resampled_merged.longitude == 1.8645)]

In [15]:
df_resampled_merged_solar1 = df_resampled_merged_solar.groupby("valid_datetime").mean().reset_index()

In [16]:
df_resampled_merged_solar1.columns

Index(['valid_datetime', 'latitude', 'longitude', 'Temperature', 'WindSpeed',
       'WindSpeed:100', 'WindDirection:100', 'CloudCover', 'RelativeHumidity',
       'PressureReducedMSL', 'SolarDownwardRadiation', 'TotalPrecipitation',
       'timestamp_utc', 'generation_mw', 'installed_capacity_mwp',
       'capacity_mwp'],
      dtype='object')

In [17]:
def set_up_solar_features(df):
    df["hour"] = df.valid_datetime.dt.hour
    df["day_of_year"] = df.valid_datetime.dt.dayofyear
    df["cos_day_of_year"] = np.cos(2 * np.pi * df.day_of_year / 365)
    df["cos_hour"] = np.cos(2 * np.pi * df.hour / 24)
    df["Mean_SolarDownwardRadiation"] = df.SolarDownwardRadiation
    df["Mean_Temperature"] = df.Temperature
    df["Std_Temperature"] = df_resampled_merged_solar.groupby("valid_datetime").std().reset_index().Temperature
    df["SolarDownwardRadiation_RW_Mean_30min"] = df.Mean_SolarDownwardRadiation.rolling(window=1, min_periods=1).mean()
    df["SolarDownwardRadiation_RW_Mean_1hour"] = df.Mean_SolarDownwardRadiation.rolling(window=2, min_periods=1).mean()
    df["SolarDownwardRadiation_dwd_Mean_Lag_30min"] = df.Mean_SolarDownwardRadiation.shift(1)
    df["SolarDownwardRadiation_dwd_Mean_Lag_1h"] = df.Mean_SolarDownwardRadiation.shift(2)
    df["SolarDownwardRadiation_dwd_Mean_Lag_24h"] = df.Mean_SolarDownwardRadiation.shift(48)
    return df

In [18]:
df_resampled_merged_solar2 = set_up_solar_features(df_resampled_merged_solar1)

In [19]:
df_resampled_merged_solar2

,valid_datetime,latitude,longitude,Temperature,WindSpeed,WindSpeed:100,WindDirection:100,CloudCover,RelativeHumidity,PressureReducedMSL,...,cos_day_of_year,cos_hour,Mean_SolarDownwardRadiation,Mean_Temperature,Std_Temperature,SolarDownwardRadiation_RW_Mean_30min,SolarDownwardRadiation_RW_Mean_1hour,SolarDownwardRadiation_dwd_Mean_Lag_30min,SolarDownwardRadiation_dwd_Mean_Lag_1h,SolarDownwardRadiation_dwd_Mean_Lag_24h
0,2024-10-07 00:00:00+00:00,52.381681,0.650333,14.12700,4.818000,8.3690,226.31250,0.893000,93.613000,99466.75350,...,0.124479,1.000000,0.000,14.12700,0.424575,0.000,0.000,NaN,NaN,NaN
1,2024-10-07 00:30:00+00:00,52.381681,0.650333,13.96225,4.939500,8.5185,232.54025,0.898250,92.788750,99514.00425,...,0.124479,1.000000,0.000,13.96225,0.439088,0.000,0.000,0.000,NaN,NaN
2,2024-10-07 01:00:00+00:00,52.381681,0.650333,13.79750,5.061000,8.6680,238.76800,0.903500,91.964500,99561.25500,...,0.124479,0.965926,0.000,13.79750,0.464677,0.000,0.000,0.000,0.000,NaN
3,2024-10-07 01:30:00+00:00,52.381681,0.650333,13.53700,5.072250,8.7065,239.91850,0.861750,91.315500,99557.52450,...,0.124479,0.965926,0.000,13.53700,0.514471,0.000,0.000,0.000,0.000,NaN
4,2024-10-07 02:00:00+00:00,52.381681,0.650333,13.27650,5.083500,8.7450,241.06900,0.820000,90.666500,99553.79400,...,0.124479,0.866025,0.000,13.27650,0.595874,0.000,0.000,0.000,0.000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,2024-10-13 10:00:00+00:00,52.381681,0.650333,10.17950,1.444667,2.5860,271.66600,0.992667,80.696667,101807.58400,...,0.226116,-0.866025,261.971,10.17950,0.450144,261.971,261.971,261.971,261.971,254.560667
309,2024-10-13 10:30:00+00:00,52.381681,0.650333,10.77075,1.377750,2.2880,268.87000,0.994000,78.741500,101822.24975,...,0.226116,-0.866025,261.971,10.77075,0.405603,261.971,261.971,261.971,261.971,244.286500
310,2024-10-13 11:00:00+00:00,52.381681,0.650333,11.36200,1.310833,1.9900,266.07400,0.995333,76.786333,101836.91550,...,0.226116,-0.965926,261.971,11.36200,0.371870,261.971,261.971,261.971,261.971,234.012333
311,2024-10-13 11:30:00+00:00,52.381681,0.650333,11.95325,1.243917,1.6920,263.27800,0.996667,74.831167,101851.58125,...,0.226116,-0.965926,261.971,11.95325,0.352064,261.971,261.971,261.971,261.971,223.738167


In [20]:
def pv_temperature_efficiency(irradiance, ambient_temp, NOCT=45, wind_speed=1, eta_0=0.18, beta=0.004):
    # Calculate cell temperature using the simplified NOCT model
    Tc = ambient_temp + (NOCT - 20) * (irradiance / 800)
    
    # Calculate the efficiency loss due to increased cell temperature
    efficiency = eta_0 * (1 - beta * (Tc - 25))
    
    return Tc, efficiency

In [22]:
df_resampled_merged_solar2["Panel_Temperature_dwd_mean"], df_resampled_merged_solar2["Panel_Efficiency_dwd_mean"] = pv_temperature_efficiency(df_resampled_merged_solar2["Mean_SolarDownwardRadiation"], df_resampled_merged_solar2["Mean_Temperature"])

In [24]:
df_resampled_merged_solar2["Panel_Temperature_dwd_std"] = df_resampled_merged_solar2.Panel_Temperature_dwd_mean.std()
df_resampled_merged_solar2["Panel_Efficiency_dwd_std"] = df_resampled_merged_solar2.Panel_Efficiency_dwd_mean.std()

In [25]:
df_resampled_merged_solar2

,valid_datetime,latitude,longitude,Temperature,WindSpeed,WindSpeed:100,WindDirection:100,CloudCover,RelativeHumidity,PressureReducedMSL,...,Std_Temperature,SolarDownwardRadiation_RW_Mean_30min,SolarDownwardRadiation_RW_Mean_1hour,SolarDownwardRadiation_dwd_Mean_Lag_30min,SolarDownwardRadiation_dwd_Mean_Lag_1h,SolarDownwardRadiation_dwd_Mean_Lag_24h,Panel_Temperature_dwd_mean,Panel_Efficiency_dwd_mean,Panel_Temperature_dwd_std,Panel_Efficiency_dwd_std
0,2024-10-07 00:00:00+00:00,52.381681,0.650333,14.12700,4.818000,8.3690,226.31250,0.893000,93.613000,99466.75350,...,0.424575,0.000,0.000,NaN,NaN,NaN,14.127000,0.187829,5.79482,0.004172
1,2024-10-07 00:30:00+00:00,52.381681,0.650333,13.96225,4.939500,8.5185,232.54025,0.898250,92.788750,99514.00425,...,0.439088,0.000,0.000,0.000,NaN,NaN,13.962250,0.187947,5.79482,0.004172
2,2024-10-07 01:00:00+00:00,52.381681,0.650333,13.79750,5.061000,8.6680,238.76800,0.903500,91.964500,99561.25500,...,0.464677,0.000,0.000,0.000,0.000,NaN,13.797500,0.188066,5.79482,0.004172
3,2024-10-07 01:30:00+00:00,52.381681,0.650333,13.53700,5.072250,8.7065,239.91850,0.861750,91.315500,99557.52450,...,0.514471,0.000,0.000,0.000,0.000,NaN,13.537000,0.188253,5.79482,0.004172
4,2024-10-07 02:00:00+00:00,52.381681,0.650333,13.27650,5.083500,8.7450,241.06900,0.820000,90.666500,99553.79400,...,0.595874,0.000,0.000,0.000,0.000,NaN,13.276500,0.188441,5.79482,0.004172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,2024-10-13 10:00:00+00:00,52.381681,0.650333,10.17950,1.444667,2.5860,271.66600,0.992667,80.696667,101807.58400,...,0.450144,261.971,261.971,261.971,261.971,254.560667,18.366094,0.184776,5.79482,0.004172
309,2024-10-13 10:30:00+00:00,52.381681,0.650333,10.77075,1.377750,2.2880,268.87000,0.994000,78.741500,101822.24975,...,0.405603,261.971,261.971,261.971,261.971,244.286500,18.957344,0.184351,5.79482,0.004172
310,2024-10-13 11:00:00+00:00,52.381681,0.650333,11.36200,1.310833,1.9900,266.07400,0.995333,76.786333,101836.91550,...,0.371870,261.971,261.971,261.971,261.971,234.012333,19.548594,0.183925,5.79482,0.004172
311,2024-10-13 11:30:00+00:00,52.381681,0.650333,11.95325,1.243917,1.6920,263.27800,0.996667,74.831167,101851.58125,...,0.352064,261.971,261.971,261.971,261.971,223.738167,20.139844,0.183499,5.79482,0.004172


In [156]:
df_resampled_merged_solar1.generation_mw.to_clipboard()

In [26]:
df_resampled_merged_solar2.generation_mw.shift(periods= 48).to_clipboard()

In [27]:
df_resampled_merged_solar2["solar_mw_lag_24h"] = df_resampled_merged_solar2.generation_mw.shift(periods= 48)

In [28]:
df_resampled_merged_solar3 = df_resampled_merged_solar2[[ 
    "Mean_SolarDownwardRadiation",
    "SolarDownwardRadiation_RW_Mean_30min",
    "SolarDownwardRadiation_RW_Mean_1hour",
    "SolarDownwardRadiation_dwd_Mean_Lag_30min",
    "SolarDownwardRadiation_dwd_Mean_Lag_1h",
    "SolarDownwardRadiation_dwd_Mean_Lag_24h",
    "Panel_Efficiency_dwd_mean",
    "Panel_Efficiency_dwd_std",
    "Panel_Temperature_dwd_mean",
    "Panel_Temperature_dwd_std",
    "Std_Temperature",
    "Mean_Temperature",
    "cos_hour",
    "cos_day_of_year","solar_mw_lag_24h","generation_mw"]]

In [29]:
df_resampled_merged_solar3.dropna(inplace=True)

C:\Users\paulh\AppData\Local\Temp\ipykernel_2168\2037698839.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resampled_merged_solar3.dropna(inplace=True)


In [30]:
df_resampled_merged_solar3

,Mean_SolarDownwardRadiation,SolarDownwardRadiation_RW_Mean_30min,SolarDownwardRadiation_RW_Mean_1hour,SolarDownwardRadiation_dwd_Mean_Lag_30min,SolarDownwardRadiation_dwd_Mean_Lag_1h,SolarDownwardRadiation_dwd_Mean_Lag_24h,Panel_Efficiency_dwd_mean,Panel_Efficiency_dwd_std,Panel_Temperature_dwd_mean,Panel_Temperature_dwd_std,Std_Temperature,Mean_Temperature,cos_hour,cos_day_of_year,solar_mw_lag_24h,generation_mw
48,0.0000,0.0000,0.00000,0.0000,0.0000,0.00000,0.188265,0.004172,13.520500,5.79482,0.555636,13.52050,1.000000e+00,0.14154,0.000000,0.000000
49,0.0000,0.0000,0.00000,0.0000,0.0000,0.00000,0.188252,0.004172,13.539250,5.79482,0.533327,13.53925,1.000000e+00,0.14154,0.000000,0.000000
50,0.0000,0.0000,0.00000,0.0000,0.0000,0.00000,0.188238,0.004172,13.558000,5.79482,0.517612,13.55800,9.659258e-01,0.14154,0.000000,0.000000
51,0.0000,0.0000,0.00000,0.0000,0.0000,0.00000,0.188266,0.004172,13.519000,5.79482,0.505375,13.51900,9.659258e-01,0.14154,0.000000,0.000000
52,0.0000,0.0000,0.00000,0.0000,0.0000,0.00000,0.188294,0.004172,13.480000,5.79482,0.496631,13.48000,8.660254e-01,0.14154,0.000000,0.000000
53,0.0000,0.0000,0.00000,0.0000,0.0000,0.00000,0.188339,0.004172,13.418500,5.79482,0.471828,13.41850,8.660254e-01,0.14154,0.000000,0.000000
54,0.0000,0.0000,0.00000,0.0000,0.0000,0.00000,0.188383,0.004172,13.357000,5.79482,0.456417,13.35700,7.071068e-01,0.14154,0.000000,0.000000
55,0.0000,0.0000,0.00000,0.0000,0.0000,0.00000,0.188406,0.004172,13.325250,5.79482,0.442051,13.32525,7.071068e-01,0.14154,0.000000,0.000000
56,0.0000,0.0000,0.00000,0.0000,0.0000,0.00000,0.188429,0.004172,13.293500,5.79482,0.432657,13.29350,5.000000e-01,0.14154,0.000000,0.000000
57,0.0000,0.0000,0.00000,0.0000,0.0000,0.00000,0.188497,0.004172,13.198750,5.79482,0.407895,13.19875,5.000000e-01,0.14154,0.000000,0.000000


In [31]:
#load scaler and model
import pickle
from sklearn.preprocessing import StandardScaler

scaler = pickle.load(open('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/paul_analyse/scaler.pkl', 'rb'))
X = df_resampled_merged_solar3.drop(columns=["generation_mw"])
X_scaled = scaler.transform(X)
Y = df_resampled_merged_solar3.generation_mw

c:\Users\paulh\anaconda3\envs\HEFTcom24\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


ValueError: X has 15 features, but StandardScaler is expecting 10 features as input.